In [2]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd

driver=webdriver.Chrome("C:/Users/pc/Main/Data_Analysis/chromedriver_win32/chromedriver.exe")
url="https://www.naver.com/"
driver.get(url)
time.sleep(3)

def naver_searching(word):
    url="https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query="+word
    return url

word="코로나현황"
url=naver_searching(word)
driver.get(url)
time.sleep(3)

def select_map(driver):
    result_map=driver.find_element_by_css_selector("#_cs_production_type > div > div:nth-child(4) > div > div.info_top > div > div > div > ul > li:nth-child(1) > a")
    result_map.click()
    time.sleep(3)

select_map(driver)
time.sleep(3)

def get_content(driver):
    html=driver.page_source
    soup=BeautifulSoup(html, "html.parser")
    areas=soup.select("div.local_info.type_increase")
    
    area_infomation_result=[]
    for i in range(len(areas)):
        area=areas[i]
        area_name=area.select("strong")[0].text #지역
        number=area.select("p")[0].text #확진자 수
        area_number = int(number.replace(",","")) #확진자 수 형변환
        area_infomation_result.append([area_name, area_number])
    
    return area_infomation_result
        
area_result=get_content(driver)

#데이터프레임 변형 및 엑셀파일 생성
import pandas as pd
area_result_df=pd.DataFrame(area_result)
area_result_df.columns=["지역","확진자 수"]
area_result_df.to_excel("C:/Users/pc/Main/Mini_project/area_info.xlsx", index=False)

##sql에 저장하기
import pymysql

conn=pymysql.connect(host="127.0.0.1", user="root",password="hp_8_8", db="choronadb", charset="utf8") #db변경필요
cur=conn.cursor()

create_table="CREATE TABLE IF NOT EXISTS chorona_kr (area char(3), area_number int)" #테이블 생성
cur.execute(create_table)

for info in area_result:
    area=info[0]
    area_num = info[1]
    sql="INSERT INTO chorona_kr VALUES ('{}', {})".format(area, area_num)
    cur.execute(sql)
    
conn.commit()
conn.close()

In [67]:
area_result

[['제주', 2494],
 ['서울', 76245],
 ['인천', 11305],
 ['광주', 3920],
 ['광주', 3920],
 ['경북', 6802],
 ['부산', 10999],
 ['대구', 13192],
 ['강원', 5242],
 ['경기', 68158],
 ['전남', 2479],
 ['충북', 4953],
 ['충남', 6547],
 ['충남', 6547],
 ['대전', 5413],
 ['세종', 944],
 ['전북', 3396],
 ['경남', 9615],
 ['울산', 3997]]